Предварительно про PyTorch:
* [Про тензоры в pytorch](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/tensor_tutorial.ipynb)
* [Про автоматическое дифференцирование и что такое .backwards()](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/autograd_tutorial.ipynb)
* [Очень простая нейронка на pytorch](https://colab.research.google.com/drive/1RsZvw4KBGn5U5Aj5Ak7OG2pHx6z1OSlF)

# Классификация текстов

## Fakenews

1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Векторизуйте.
4. Обучите на полученных векторах алгоритм классификации.

Мы уже видели как эта задача выполняется с помощью Word2vec. Давайте вспомним.

In [19]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2021-11-30 16:29:34--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253562 (1.2M) [text/plain]
Saving to: ‘Constraint_Train.csv.1’

Constraint_Train.cs 100%[===================>]   1.20M  --.-KB/s    in 0.05s   

2021-11-30 16:29:34 (23.1 MB/s) - ‘Constraint_Train.csv.1’ saved [1253562/1253562]



In [20]:
import pandas as pd

In [21]:
df = pd.read_csv('Constraint_Train.csv')

In [22]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [23]:
from nltk.tokenize import word_tokenize
from tqdm import tqdm

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
sentences = [word_tokenize(text.lower()) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:02<00:00, 2706.98it/s]


In [26]:
from gensim.models.word2vec import Word2Vec
%time model_tweets = Word2Vec(sentences, workers=4, size=400, min_count=3, window=5, iter=25)

CPU times: user 19.6 s, sys: 197 ms, total: 19.8 s
Wall time: 11.2 s


In [27]:
model_tweets.wv.most_similar('france')

[('bags', 0.8093336820602417),
 ('wenliang', 0.7785456776618958),
 ('arrest', 0.7718162536621094),
 ('floor', 0.7695581912994385),
 ('li', 0.7641429901123047),
 ('enforcing', 0.7637995481491089),
 ('deceased', 0.761199951171875),
 ('tower', 0.7581547498703003),
 ('2015', 0.7563822269439697),
 ('crying', 0.752638578414917)]

In [28]:
model_tweets.init_sims()

In [29]:
import numpy as np

In [30]:
def get_text_embedding(text):
    result = []
    for word in word_tokenize(text.lower()):
        if word in model_tweets.wv:
            result.append(model_tweets.wv[word])

    if len(result):
        result = np.sum(result, axis=0)
    else:
        result = np.zeros(400)
    return result

In [31]:
features = [get_text_embedding(text) for text in tqdm(df.tweet)]

100%|██████████| 6420/6420 [00:03<00:00, 1894.08it/s]


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)

In [34]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [35]:
from sklearn.metrics import classification_report

In [36]:
predicted = model.predict(X_test)

In [37]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.91      0.93      0.92      1032
        real       0.93      0.92      0.92      1087

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Вариант 2 с использованием *DecisionTreeClassifier*

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
model = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [40]:
predicted = model.predict(X_test)

In [41]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.88      0.84      0.86      1032
        real       0.86      0.89      0.88      1087

    accuracy                           0.87      2119
   macro avg       0.87      0.87      0.87      2119
weighted avg       0.87      0.87      0.87      2119



Вариант 3 с использованием *SVC*

In [42]:
from sklearn.svm import SVC

In [43]:
model = SVC(kernel='rbf', C=1.0, random_state=42)
model.fit(X_train, y_train)

SVC(random_state=42)

In [44]:
predicted = model.predict(X_test)

In [45]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.92      0.93      0.93      1032
        real       0.94      0.93      0.93      1087

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



Вариант 4 с использованием *RandomForestClassifier*

In [46]:
from sklearn.ensemble import RandomForestClassifier

In [47]:
model = RandomForestClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=42)

In [48]:
predicted = model.predict(X_test)

In [49]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.94      0.92      0.93      1032
        real       0.93      0.94      0.94      1087

    accuracy                           0.93      2119
   macro avg       0.93      0.93      0.93      2119
weighted avg       0.93      0.93      0.93      2119



###  Что будет, если использовать самый наивный метод?

In [50]:
from sklearn.feature_extraction.text import CountVectorizer

In [51]:
vec = CountVectorizer()

In [52]:
bow = vec.fit_transform(df.tweet)

In [53]:
X_train, X_test, y_train, y_test = train_test_split(features, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression()

In [54]:
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.90      0.94      0.92      1007
        real       0.94      0.91      0.93      1112

    accuracy                           0.92      2119
   macro avg       0.92      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



Конечно, мы всегда можем поиграться с предобработкой.

### PyTorch + LSTM

In [57]:
labels = (df.label == 'real').astype(int).to_list()

Нужно заранее задать размер для макксимальной длины предложений.

In [58]:
token_lists = [word_tokenize(text.lower()) for text in df.tweet]
max_len = len(max(token_lists, key=len))

In [59]:
max_len

1592

Это слишком много. Но какая длина обычно?

In [60]:
from collections import Counter
fd = Counter([len(tokens) for tokens in token_lists])

In [61]:
fd.most_common(10)

[(20, 179),
 (25, 174),
 (22, 170),
 (18, 170),
 (19, 167),
 (21, 167),
 (16, 162),
 (15, 161),
 (17, 161),
 (23, 157)]

Зададим максимум 400.

Возьмём те же w2v эмбеддинги.

In [62]:
def get_word_embedding(tokens, max_len):
    result = []
    for i in range(max_len):
        if i < len(tokens):
            word = tokens[i]
            if word in model_tweets.wv:
                result.append(model_tweets.wv[word])
            else:
                result.append(np.zeros(400))
        else:
            result.append(np.zeros(400))
    return result

In [63]:
features = [get_word_embedding(text, 400) for text in tqdm(token_lists)]

100%|██████████| 6420/6420 [00:08<00:00, 770.86it/s]


In [64]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33)

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim

In [66]:
len(features[0][0])

400

In [67]:
len(X_train)

4301

In [68]:
len(X_train[0])

400

In [69]:
len(X_train[0][0])

400

In [70]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.lstm = nn.LSTM(400, 100)
        self.out = nn.Linear(100, 1)

    def forward(self, x):
        embeddings, (shortterm, longterm) = self.lstm(x.transpose(0, 1))
        prediction = torch.sigmoid(self.out(longterm))
        return prediction


net = Net()
print(net)

Net(
  (lstm): LSTM(400, 100)
  (out): Linear(in_features=100, out_features=1, bias=True)
)


In [ ]:
in_data = torch.tensor(X_train).float()
targets = torch.tensor(y_train).float()

In [ ]:
in_data.shape

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
def train_one_epoch(in_data, targets, batch_size=16):
    for i in tqdm(range(0, in_data.shape[0], batch_size)):
        batch_x = in_data[i:i + batch_size]
        batch_y = targets[i:i + batch_size]
        optimizer.zero_grad()
        output = net(batch_x)
        loss = criterion(output.reshape(-1), batch_y)
        loss.backward()
        optimizer.step()
    print(loss)

In [ ]:
train_one_epoch(in_data, targets)

Что получилось?

In [ ]:
in_data_test = torch.tensor(X_test).float()
targets_test = torch.tensor(y_test).float()

In [ ]:
with torch.no_grad():
    output = net(in_data_test).reshape(-1)

In [ ]:
result = (output > 0.5) == targets_test

In [ ]:
result.sum().item() / len(result)

Но такую модель надо учить дольше(